In [ ]:
#https://arxiv.org/abs/2409.17745

## Import required libraries and classes

In [ ]:
import pyterrier as pt
import configuration
from models.llm_tokenizer import LoadLLM_Model, LoadLLM_Tokenizer
from models.llm_generator import LLMGenerator
from tqdm import tqdm
import torch

In [ ]:
'''
SEEDING FOR GETTING SAME OUTPUT
'''

torch.manual_seed(42)

## Import the dataset

In [ ]:
ds = 'dl19'
dataset = pt.datasets.get_dataset(configuration.datasets[ds]['name'])
topics = dataset.get_topics(configuration.datasets[ds]['topics'])
qrels = dataset.get_qrels()

In [ ]:
topics = topics.head(4)

## Load Index

In [ ]:
index = pt.IndexFactory.of(configuration.datasets[ds]['index'], memory=True)
meta = index.getMetaIndex()

## Initialize the LLM - Model and Tokenizer

In [ ]:
#llm = 'flanxl'
model_name = 'zephyr'
model = LoadLLM_Model(model_name)
tokenizer = LoadLLM_Tokenizer(model_name)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
parameters = {
    'device': device,
    'model': model,
    'tokenizer': tokenizer,
    'enc_tokens': 0,
}

## Get started with zero-shot prompting

In [ ]:
working_data = topics.copy()

In [ ]:
working_data.head()

In [ ]:
'''

FIND THE ACTUAL RELEVANT ANSWERS

'''

In [ ]:
working_data["rel_docno"] = ''
working_data["text"] = ''

In [ ]:
for index,row in tqdm(working_data.iterrows(), total=working_data.shape[0]):
    qid = row['qid']
    # find rel_doc_no
    rel_docs = qrels.loc[(qrels['qid']==qid) & (qrels['label']>=2)]
    rel_docs = rel_docs.sort_values('label', ascending=False)
    rel_doc_id = rel_docs.values[0][1]
    working_data.loc[index, 'rel_docno'] = rel_doc_id

In [ ]:
for index,row in tqdm(working_data.iterrows(), total=working_data.shape[0]):
    doc_no = row['rel_docno']
    working_data.loc[index, 'text'] = meta.getItem("text", int(doc_no))

In [ ]:
working_data.head()

In [ ]:
'''

ENGINEER THE PROMPT

'''

In [ ]:
template_prompt = '''
    Answer the following query in detail with explanations.
    Query: "{QUERY}"
    Answer:'''

In [ ]:
'''

GENERATE ZERO-SHOT ANSWER FROM THE MODEL

'''

In [ ]:
for index,row in tqdm(working_data.iterrows(), total=working_data.shape[0]):
    qid = row['qid']
    query = row['query']
    prompt = template_prompt.format(QUERY=query)
    gen_text = LLMGenerator.generate(model_name, parameters, prompt, gen_mode = True)
    if model_name=='zephyr':
        llm_output = gen_text.split('\n')
        working_data.loc[index, 'llm_text'] = llm_output[len(llm_output)-1]
    else:
        working_data.loc[index, 'llm_text'] = gen_text

In [ ]:
working_data

In [ ]:
'''

EVALUATE THE GENERATED OUTPUT

'''

In [ ]:
## Analyse answer quality
import os
from sentence_transformers import SentenceTransformer, util

cache_path = os.path.join(os.path.expanduser('~'), 'Documents', 'Nilanjan', 'cache')
os.environ['TRANSFORMERS_CACHE'] = cache_path
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
for index,row in tqdm(working_data.iterrows(), total=working_data.shape[0]):
    text = row['text']
    llm_text = row['llm_text']
    
    embedding_text = model.encode(text)
    embedding_llm_text = model.encode(llm_text)
    
    sim_score = util.pytorch_cos_sim(embedding_text, embedding_llm_text)
    sim_score = sim_score.tolist()
    sim_score = sim_score[0]

    working_data.loc[index, 'sim_score'] = sim_score

In [ ]:
working_data

## Get started with few-shot ICL prompts

In [ ]:
icl_data = working_data[['qid','query','rel_docno','text']]

In [ ]:
'''

ENGINEER THE PROMPT

'''

In [ ]:
template_icl_prompt = '''
    Answer the following query in detail with explanations.
    
    Query: "{EX_QUERY}"
    Answer: "{EX_ANSWER}"

    Query: "{QUERY}"
    Answer:'''

In [ ]:
'''

GENERATE USING ICL EXAMPLES

'''

In [ ]:
ex_num = 0

for index,row in tqdm(icl_data.iterrows(), total=icl_data.shape[0]):
    qid = row['qid']
    query = row['query']
    ex_query = row['query']
    ex_answer = row['text']
    prompt = template_icl_prompt.format(EX_QUERY=ex_query, EX_ANSWER=ex_answer, QUERY=query)
    gen_text = LLMGenerator.generate(model_name,parameters, prompt, gen_mode = True)
    if model_name=='zephyr':
        llm_output = gen_text.split('\n')
        icl_data.loc[index, 'llm_text'] = llm_output[len(llm_output)-1]
    else:
        icl_data.loc[index, 'llm_text'] = gen_text

In [ ]:
'''

EVALUATE THE GENERATED OUTPUT

'''

In [ ]:
for index,row in tqdm(icl_data.iterrows(), total=icl_data.shape[0]):
    text = row['text']
    llm_text = row['llm_text']
    
    embedding_text = model.encode(text)
    embedding_llm_text = model.encode(llm_text)
    
    sim_score = util.pytorch_cos_sim(embedding_text, embedding_llm_text)
    sim_score = sim_score.tolist()
    sim_score = sim_score[0]

    icl_data.loc[index, 'sim_score'] = sim_score

In [ ]:
icl_data